# Extract daily flows from HYDAT

In [1]:
import os
import sys
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
sys.path.append(str(Path().absolute().parent))
import python_cs_functions as cs

### Config handling

In [2]:
# Specify where the config file can be found
config_file = '../0_config/config.txt'

In [3]:
# Get the required info from the config file
data_path = cs.read_from_config(config_file,'data_path')

# CAMELS-spat metadata
cs_meta_path = cs.read_from_config(config_file,'cs_basin_path')
cs_meta_name = cs.read_from_config(config_file,'cs_meta_name')
cs_unusable_name = cs.read_from_config(config_file,'cs_unusable_name')

# Basin folder
cs_basin_folder = cs.read_from_config(config_file, 'cs_basin_path')
basins_path = Path(data_path) / cs_basin_folder

# Data period
time_s = datetime.strptime( cs.read_from_config(config_file, 'hydat_start_t'), '%Y-%m-%d')
time_e = datetime.strptime( cs.read_from_config(config_file, 'hydat_start_e'), '%Y-%m-%d')

# Hydat folder
ref_path = cs.read_from_config(config_file,'ref_shps_path')

### Data loading

In [4]:
# CAMELS-spat metadata file
cs_meta_path = Path(data_path) / cs_meta_path
cs_meta = pd.read_csv(cs_meta_path / cs_meta_name)

In [5]:
# Open list of unusable stations; Enforce reading IDs as string to keep leading 0's
cs_unusable = pd.read_csv(cs_meta_path / cs_unusable_name, dtype={'Station_id': object}) 

### Define data location

In [7]:
# Construct the download location
hydat_path = Path(data_path) / ref_path / 'RHBN-CAN'

In [8]:
# Find the HYDAT database. Assumes only 1 sqlite3 database exists
hydat_name = sorted(hydat_path.glob('*.sqlite3'))[0]

In [9]:
# open database
db = cs.connect_to_sqlite_database(hydat_path/hydat_name)

### Loop over sites and extract the flow record

In [10]:
# Prepare the metadata file
meta_column_start = 'dv_flow_obs_availability_start'
meta_column_end   = 'dv_flow_obs_availability_end'
meta_column_miss  = 'flow_obs_missing_daily'
c_start = np.where(cs_meta.columns == meta_column_start)[0][0]
c_end   = np.where(cs_meta.columns == meta_column_end)[0][0]
c_miss  = np.where(cs_meta.columns == meta_column_miss)[0][0]

In [11]:
# Preliminaries
sql_table = 'DLY_FLOWS'
sql_field = 'STATION_NUMBER'
sbs = [] # We'll store QC symbols in here and check them later
dnf = [] # We'll store a check here to see if we have any data at all available here

In [12]:
# Loop over the Canada stations only
dnf = [] # List of incomplete stations, retaining these for easier printout and checking later
for ix,row in cs_meta.iterrows():
    if row.Country == 'CAN':
        
        # Get paths, etc
        site, _, _, raw_path_dv, _,_ = cs.prepare_flow_download_outputs(cs_meta, ix, basins_path, time='daily')
        
        # Resume after interupts
        if os.path.isfile(raw_path_dv): # If raw file exists it must have been created already 
            continue
                
        # Construct the SQL query we need
        query = cs.construct_query_from_dataframe_column(sql_table, sql_field, [site])
        
        # Get the daily flow data from the HYDAT database as a dataframe
        station_table = cs.sql_query_to_dataframe(db, query)
        
        # Restructure the daily flows into a mangeable time series
        df = cs.restructure_daily_flows_in_hydat_table(station_table, time_s, time_e, site)
        
        # Abort and track if we didn't find any data
        if len(df) == 0:
            dnf.append(site)
            print(f'No data found for site {site}')
            continue
        
        # Update meta data
        cs_meta.iat[ix,c_start] = df.index[0].strftime('%Y-%m-%d %X')
        cs_meta.iat[ix,c_end]   = df.index[-1].strftime('%Y-%m-%d %X')
        cs_meta.iat[ix,c_miss]  = df['FLOW'].isna().sum()
        
        # Save to file
        df.to_csv(raw_path_dv)
        
        # Track the symbols for (possible) later use
        sbs.append(df['SYMBOL'].unique())
        print(f'Finished site {site}')

No data found for site 05RE002
No data found for site 06DA001
No data found for site 07BJ006
No data found for site 07QD002
No data found for site 07SB017
No data found for site 08AA007
No data found for site 08KH011
No data found for site 08LD003
No data found for site 08LF023
No data found for site 08MG020
No data found for site 08MG022
No data found for site 09AA004
No data found for site 09AE002


In [23]:
# Save the metadata file
cs_meta.to_csv(cs_meta_path / cs_meta_name, encoding='utf-8', index=False)

## Check sites for which we could not obtain flow data

In [12]:
# Print which basins we need to check
for entry in dnf:
    print(f'No daily data extracted for gauge {entry}')
print('End of list')

No daily data extracted for gauge 05RE002
No daily data extracted for gauge 06DA001
No daily data extracted for gauge 07BJ006
No daily data extracted for gauge 07QD002
No daily data extracted for gauge 07SB017
No daily data extracted for gauge 08AA007
No daily data extracted for gauge 08KH011
No daily data extracted for gauge 08LD003
No daily data extracted for gauge 08LF023
No daily data extracted for gauge 08MG020
No daily data extracted for gauge 08MG022
No daily data extracted for gauge 09AA004
No daily data extracted for gauge 09AE002
End of list


Manual checks using WSC website indicate that for these stations only water levels, but not flows, are available: https://wateroffice.ec.gc.ca/search/historical_e.html
- 05RE002: `WEAVER LAKE AT OUTLET` No flows, but level data available for 1967-2021
- 06DA001: `WOLLASTON LAKE AT ROSS CHANNEL` No flows, but level data available for 1971-2022
- 07BJ006: `LESSER SLAVE LAKE AT SLAVE LAKE` No flows, but level data available for 1979-2021
- 07QD002: `NONACHO LAKE NEAR LUTSELK'E (SNOWDRIFT)` No flows, but level data available for 1962-2022
- 07SB017: `PRELUDE LAKE NEAR YELLOWKNIFE` No flows, but level data available for 1995-2022
- 08AA007: `SEKULMUN LAKE NEAR WHITEHORSE` No flows, but level data available for 1980-2021
- 08KH011: `QUESNEL LAKE NEAR LIKELY` No flows, but level data available for 1956-2022
- 08LD003: `ADAMS LAKE NEAR SQUILAX` No flows, but level data available for 1949-2021
- 08LF023: `THOMPSON RIVER AT KAMLOOPS` Flow data for 1911-1913 only, level data available for 1911-2022
- 08MG020: `LILLOOET LAKE NEAR PEMBERTON` No flows, but level data available for 1971-2021
- 08MG022: `HARRISON RIVER BELOW MORRIS CREEK` No flows, but level data available for 1973-2021
- 09AA004: `BENNETT LAKE AT CARCROSS` No flows, but level data available for 1947-2022
- 09AE002: `TESLIN LAKE AT TESLIN` No flows, but level data available for 1944-2019

### Update the metadata file

In [13]:
country = 'CAN'

In [14]:
missing = 'dv'
reason = 'No discharge values available (only water levels)'

In [16]:
cs_unusable = pd.concat([cs_unusable, pd.DataFrame({'Country': country,
                                                    'Station_id': dnf,
                                                    'Missing': missing,
                                                    'Reason': reason})])

In [20]:
cs_unusable = cs_unusable.reset_index(drop=True)

In [21]:
cs_unusable.to_csv(cs_meta_path / cs_unusable_name, encoding='utf-8', index=False)

## Check which streamflow codes we have

In [94]:
# Print which data codes we have in the data
sbs_flat = []
for arr in sbs:
    for item in arr:
        if str(item) != 'nan':
            sbs_flat.append(item)

In [95]:
set(sbs_flat)

{' ', 'A', 'B', 'E', None}

In [ ]:
# For posterity
sbs_flat = [' ', 'A', 'B', 'E', None]

In [100]:
cs.find_table_contents(db,'DATA_SYMBOLS',to_screen=True);

('A', 'Partial Day', 'Journée incomplète')
('B', 'Ice Conditions', 'Conditions à glace')
('D', 'Dry', 'Sec')
('E', 'Estimated', 'Estimé')
('S', 'Sample(s) collected this day', 'échantillons prélevés ce jour-là')


### Database investigation

In [17]:
# list the available tables
tables = cs.find_all_table_names(db) # -> we want 'STATIONS'

('STATIONS',)
('CONCENTRATION_SYMBOLS',)
('SED_SAMPLES_PSD',)
('ANNUAL_INSTANT_PEAKS',)
('STN_DATUM_UNRELATED',)
('DATA_SYMBOLS',)
('SED_VERTICAL_LOCATION',)
('STN_DATA_COLLECTION',)
('PEAK_CODES',)
('SED_DATA_TYPES',)
('MEASUREMENT_CODES',)
('SED_VERTICAL_SYMBOLS',)
('DATA_TYPES',)
('DLY_FLOWS',)
('STN_REMARKS',)
('STN_DATUM_CONVERSION',)
('AGENCY_LIST',)
('SED_DLY_SUSCON',)
('STN_OPERATION_SCHEDULE',)
('STN_DATA_RANGE',)
('PRECISION_CODES',)
('SED_DLY_LOADS',)
('DLY_LEVELS',)
('OPERATION_CODES',)
('STN_REGULATION',)
('DATUM_LIST',)
('ANNUAL_STATISTICS',)
('VERSION',)
('REGIONAL_OFFICE_LIST',)
('SAMPLE_REMARK_CODES',)
('STN_REMARK_CODES',)
('SED_SAMPLES',)
('STN_STATUS_CODES',)


In [18]:
# Find the headers in the DLY_FLOWS table
table_name = 'DLY_FLOWS'
headers = cs.find_table_headers(db,table_name) # -> we need to subset on 'STATION_NUMBER'

['STATION_NUMBER', 'YEAR', 'MONTH', 'FULL_MONTH', 'NO_DAYS', 'MONTHLY_MEAN', 'MONTHLY_TOTAL', 'FIRST_DAY_MIN', 'MIN', 'FIRST_DAY_MAX', 'MAX', 'FLOW1', 'FLOW_SYMBOL1', 'FLOW2', 'FLOW_SYMBOL2', 'FLOW3', 'FLOW_SYMBOL3', 'FLOW4', 'FLOW_SYMBOL4', 'FLOW5', 'FLOW_SYMBOL5', 'FLOW6', 'FLOW_SYMBOL6', 'FLOW7', 'FLOW_SYMBOL7', 'FLOW8', 'FLOW_SYMBOL8', 'FLOW9', 'FLOW_SYMBOL9', 'FLOW10', 'FLOW_SYMBOL10', 'FLOW11', 'FLOW_SYMBOL11', 'FLOW12', 'FLOW_SYMBOL12', 'FLOW13', 'FLOW_SYMBOL13', 'FLOW14', 'FLOW_SYMBOL14', 'FLOW15', 'FLOW_SYMBOL15', 'FLOW16', 'FLOW_SYMBOL16', 'FLOW17', 'FLOW_SYMBOL17', 'FLOW18', 'FLOW_SYMBOL18', 'FLOW19', 'FLOW_SYMBOL19', 'FLOW20', 'FLOW_SYMBOL20', 'FLOW21', 'FLOW_SYMBOL21', 'FLOW22', 'FLOW_SYMBOL22', 'FLOW23', 'FLOW_SYMBOL23', 'FLOW24', 'FLOW_SYMBOL24', 'FLOW25', 'FLOW_SYMBOL25', 'FLOW26', 'FLOW_SYMBOL26', 'FLOW27', 'FLOW_SYMBOL27', 'FLOW28', 'FLOW_SYMBOL28', 'FLOW29', 'FLOW_SYMBOL29', 'FLOW30', 'FLOW_SYMBOL30', 'FLOW31', 'FLOW_SYMBOL31']


In [20]:
# Check the contents of a single row to find which format we need to specify STATION_NUMBER in
contents = cs.find_table_contents(db,table_name,to_screen=False)
contents[0] # -> we need to specify station IDs as '01AA002' INCLUDING the apostrophes

('01AD001',
 1928,
 10,
 1,
 31,
 19.799999237060547,
 613.2000122070312,
 6,
 15.300000190734863,
 26,
 29.700000762939453,
 16.600000381469727,
 'E',
 16.600000381469727,
 'E',
 16.600000381469727,
 'E',
 16.600000381469727,
 'E',
 16.600000381469727,
 'E',
 15.300000190734863,
 'E',
 15.300000190734863,
 'E',
 15.300000190734863,
 'E',
 15.300000190734863,
 'E',
 15.300000190734863,
 'E',
 15.899999618530272,
 'E',
 15.899999618530272,
 'E',
 15.899999618530272,
 'E',
 15.899999618530272,
 'E',
 15.899999618530272,
 'E',
 15.300000190734863,
 'E',
 15.300000190734863,
 'E',
 15.300000190734863,
 'E',
 15.300000190734863,
 'E',
 15.300000190734863,
 'E',
 23.899999618530277,
 'E',
 23.899999618530277,
 'E',
 23.899999618530277,
 'E',
 23.899999618530277,
 'E',
 23.899999618530277,
 'E',
 29.700000762939453,
 'E',
 29.700000762939453,
 'E',
 29.700000762939453,
 'E',
 29.700000762939453,
 'E',
 29.700000762939453,
 'E',
 29.700000762939453,
 'E')